### Decorators
A **decorator** is a function that takes a function as an argument and returns a function as a return value.

Here is an example:

```python
def substitute(a_function):
    def new_function(*args, **kwargs):
        return "I'm not that other function"
    return new_function
```

Okay, but what does this *actually* mean?

A **decorator** is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

Hmmm, I think this is getting closer to something we can use.

What would be some use cases?

Some uses include:
* logging,
* enforcing access control and authentication,
* instrumentation and timing functions,
* rate-limiting,
* caching; and more.

It's helpful to know that functions are **first-class objects** in Python. This means that functions can be passed around, and used as arguments, just like any other value (e.g, string, int, float).

A **decorator** decorates another function. For example, assuming an existing decorator named *decorate*, this code:

```python
@decorate
def target():
    print('running target()')
```
Has the same effect as writing this:

```python
def target():
    print('running target()')
    
target = decorate(target)
```

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


The flow here is:
1. deco returns its inner function object
2. target is decorated by deco
3. Invoking the decorated target actually runs inner
4. Inspection reveals that target is now a reference to inner

#### Gimme something I would acutally use!
Most of at some point or another want to see how fast our code is running. Decorators to the rescue!

In [6]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12315702s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000049s] factorial(1) -> 1
[0.00002935s] factorial(2) -> 2
[0.00005129s] factorial(3) -> 6
[0.00007181s] factorial(4) -> 24
[0.00009215s] factorial(5) -> 120
[0.00011333s] factorial(6) -> 720
6! =  720


#### Stacked Decorators
We can decorate with more than one decorator at a time. For example:

```python
@d1
@d2
def f():
    print('f')
```
Is the same as:

```python 
def f():
    print('f')

f = d1(d2(f))
```

#### Parameterized Decorators


In [2]:
import time 
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12324238s] snooze(0.123) -> None
[0.12320256s] snooze(0.123) -> None
[0.12323046s] snooze(0.123) -> None


Let's try changing that format

In [11]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12318015098571777s
snooze: 0.12315487861633301s
snooze: 0.12314939498901367s


or

In [13]:
@clock('{name}({args}) dt = {elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt = 0.123s
snooze(0.123) dt = 0.123s
snooze(0.123) dt = 0.123s
